In [2]:
#import packages and formatting statements

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join, Column
from astropy.wcs import WCS
from astropy.io import ascii
import os

plt.rc('text',usetex=False)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

In [4]:
# Compile galaxy and OSC objects
DataFile = '../Data/1.ProjectTable.csv'
Data = Table.read(DataFile, format='csv') 
galaxies, JWSTmaps = Data["#galaxy"], Data["JWST_file_name"]
SNe = np.loadtxt('../Data/1.OSCRaDeg.txt', dtype = str, unpack=True, usecols = 0)
ras = np.loadtxt('../Data/1.OSCRaDeg.txt',delimiter='" ', unpack = True, usecols=1)
decs = np.loadtxt('../Data/1.OSCDecDeg.txt',delimiter='" ',unpack = True, usecols = (1))
types = np.loadtxt('../Data/1.OSCCleaned.csv', dtype=str, unpack=True, usecols=3)

In [5]:
#Checks if Supernovae are in an image and reports back their x & y coords if in map

def check_in_image(SNras, SNdecs, SNnames, SNtypes, image):
    
    hdulist  = pyfits.open(image)
    

    map = hdulist[1].data
    #emap = "find a way to use RMS from galaxy for these"
    wcs = WCS(hdulist[1].header, naxis=2)
    #beamsize = hdulist[0].header["BEAM"]
    naxis = wcs._naxis #size of image
    naxis1 = hdulist[1].header['NAXIS1']
    naxis2 = hdulist[1].header['NAXIS2']
        
    coords_arr = np.column_stack((SNras, SNdecs)) # ras and decs now [ra,dec]
    pix_x, pix_y = wcs.wcs_world2pix(SNras,SNdecs,0)

    #use world coordinates of all SNe to see if any fall in image (our version of footprint_contains)

    is_in_x = (pix_x >= 0) & (pix_x <= naxis[0]-1) #because of 0-indexing
    is_in_y = (pix_y >= 0) & (pix_y <= naxis[1]-1)
             
    #get the name, ra, and dec of the SNe that fall in image
    #boolean array indexing (gives back array of Trues and Falses)
    #we are pulling out the SNe that are True and assigning them to own arrays
    name_in_image = np.array(SNnames)[is_in_x & is_in_y]
    type_in_image = np.array(SNtypes)[is_in_x & is_in_y]
    ra_in_image = np.array(SNras)[is_in_x & is_in_y]
    dec_in_image = np.array(SNdecs)[is_in_x & is_in_y]

    x_coord = np.array(pix_x)[is_in_x & is_in_y]
    y_coord = np.array(pix_y)[is_in_x & is_in_y]
    
    return(name_in_image, type_in_image, ra_in_image, dec_in_image, x_coord, y_coord, map, naxis1, naxis2)
    #print(name_in_image, type_in_image, ra_in_image, dec_in_image, x_coord, y_coord)


In [6]:
# Build JWST Catalogue That has all OSC entries within PHANGS-MUSE footprint

with open('../Data/2.SNeJWST_Cat.txt', 'w') as text_file:
    print('#Galaxy,Supernova,Ra,Dec,xCoord,yCoord,Type,Intensity', file = text_file)

for i in range(len(galaxies)):
    if (JWSTmaps[i] != "" and os.path.isfile(JWSTmaps[i])):
    
        found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, naxis1, naxis2 = check_in_image(ras, decs, SNe, types, JWSTmaps[i])
        intensity = []
    
        for k in range(len(found_type)):

            xVal = int(round(x_coord[k]))
            yVal = int(round(y_coord[k]))
            if (yVal <= naxis2) and (xVal <= naxis1):
                intensity.append(map[yVal, xVal])

            else: 
                intensity.append(np.nan)


        with open('../Data/2.SNeJWST_Cat.txt', 'a') as text_file:
            for j in range(len(found_name)):
                print(galaxies[i], ",", found_name[j],",", found_ra[j], ",",found_dec[j],",", x_coord[j],",",y_coord[j],",", found_type[j],",", intensity[j], file = text_file)


In [7]:
# Turn files into data structure and clean

JWSTdataFile = '../Data/2.SNeJWST_Cat.txt'
database = Table.read(JWSTdataFile, format='ascii') 
database = join(database, Data, keys=('Galaxy'))

#database
ascii.write(database, '../Data/2.SNeJWSTMasterCat.csv', format='csv', fast_writer=False, overwrite=True)  


IndexError: list index out of range

In [7]:
print(database[-1])

 Galaxy Supernova         Ra               Dec             ESO_xCoord         ESO_yCoord    Type ESO_Intensity MUSE_xCoord MUSE_yCoord MUSE_Intensity                               ESO-MPG_Hasub_wcomb_corr                               MUSEResolution MUSEmap
------- --------- ----------------- ------------------ ------------------ ----------------- ---- ------------- ----------- ----------- -------------- ------------------------------------------------------------------------------------ -------------- -------
NGC6744  SN2005at 287.4732777777778 -63.82312037037037 3720.2351509629934 4471.392323277116   Ic      2.962026          --          --             -- /home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC6744_Hasub_flux_corrs.fits             --      --


In [8]:

# SN_name = [x[2:-2] for x in database["Supernova"]]

# # Here we are going to filter out our undesired OSC entries:
#     # One is a double counted SN named "PTSS-19clju", any LRNs or LBVs or IIns, and any candidates

# flags = []
# flag1 = database['Supernova'] == "PTSS-19clju" 
# flag3 = database['Supernova'] == "Gaia21fml" 
# flag5 = database['Supernova'] == "Gaia21cum"

# flag2 = []

# for i in range(len(SN_name)):
#     if ("LRN" in database['Type'][i] or "LBV" in database['Type'][i] or "IIn" in database['Type'][i] or "Candidate" in database['Type'][i]):
#         flag2.append(True)
#     else:
#         flag2.append(False)

# for i in range(len(flag1)):
#     if flag1[i] == True or flag2[i] == True or flag3[i] == True or flag5[i] == True:
#         flags.append(True)
#     else:
#         flags.append(False)
        
# flagsArr = np.array(flags)

# c = Column(data=flags, name='Removed')
# database.add_column(c)
# ascii.write(database, '../Data/2.SNeHaMasterCat.csv', format='csv', fast_writer=False, overwrite = True) 

# cleanData = database[flagsArr==False]

# cleanData

# ascii.write(cleanData, '../Data/2.SNeHaMasterCatClean.csv', format='csv', fast_writer=False, overwrite = True)  

In [9]:
cleanData

Galaxy,Supernova,Ra,Dec,ESO_xCoord,ESO_yCoord,Type,ESO_Intensity,MUSE_xCoord,MUSE_yCoord,MUSE_Intensity,ESO-MPG_Hasub_wcomb_corr,MUSEResolution,MUSEmap,Removed
str8,str23,float64,float64,float64,float64,str12,float64,float64,float64,float64,str85,float64,str80,bool
NGC0628,PS15blm,24.258708333333335,15.777000000000001,1253.8697326978763,2391.66186884759,unclassified,6.3913784,--,--,--,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC0628-0.92asec_MAPS.fits,False
NGC0628,SN2002ap,24.09940416666667,15.753649074074074,3577.0564594599123,2037.1622175556208,Ic,-0.17532486,--,--,--,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC0628-0.92asec_MAPS.fits,False
NGC0628,SN2003gd,24.17770555555555,15.738966666666665,2434.990386106029,1814.759548435055,II,0.55219537,--,--,--,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC0628-0.92asec_MAPS.fits,False
NGC0628,SN2013ej,24.200666666666667,15.758611111111112,2100.1951571946192,2112.559714338542,II,0.09700711,243.19103784243157,305.29329959411643,nan,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC0628_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC0628-0.92asec_MAPS.fits,False
NGC1087,SN1995V,41.61152777777777,-0.4987861111111111,648.7610232419815,747.9909198104285,II,5.0002465,183.09993631014947,450.72004174402633,2663.371834437333,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1087_Hasub_flux_corrs.fits,0.92,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1087-0.92asec_MAPS.fits,False
NGC1300,LSQ14dtr,49.96679166666666,-19.478694444444443,841.190251571016,472.6653386793862,Ia,-0.06859025,--,--,--,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1300_Hasub_flux_corrs.fits,0.89,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1300-0.89asec_MAPS.fits,False
NGC1365,SN1983V,53.381876388888884,-36.14859166666667,3270.6237389153885,2906.610095778981,Ic,9.090737,1027.9095105186898,309.74396493992725,5743.3576143866185,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits,1.15,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1365-1.15asec_MAPS.fits,False
NGC1365,SN1957C,53.38349999999999,-36.11770833333333,3250.4029666345923,3374.662468105187,unclassified,2.834359,1004.4155369313344,865.6486483514018,1697.5272118916866,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits,1.15,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1365-1.15asec_MAPS.fits,False
NGC1365,SN2001du,53.3713125,-36.142111111111106,3399.846121958709,3004.9164893583365,II,1.5359482,1181.490382249507,426.35406918108174,929.5770449215998,/home/mayker.1/Desktop/SNeHaLargeData/HaSUB_wcomb_corr/NGC1365_Hasub_flux_corrs.fits,1.15,/data/fourier/sun.1608/PHANGS/MUSE/DR2.1/MUSEDAP/copt/NGC1365-1.15asec_MAPS.fits,False


In [12]:
# Plot MUSE Galaxy Files with SNe
inMuse = []

for i in range(len(data["Galaxy"])):
    
    if (data["MUSEmap"][i] != "" and os.path.isfile(data["MUSEmap"][i])):     
    
        found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, naxis1, naxis2 = check_in_image(data['Ra'], data['Dec'], data['Supernova'], data['Type'], data["MUSEmap"][i])
    
        plt.figure(figsize=(7, 7))
        logmap = np.log10(map)
        plt.imshow(logmap, origin='lower', interpolation='nearest', zorder=1)

        for k in range(len(found_name)):
            inMuse.append(found_name[k])
            string = found_name[k]

            if (len(found_type[k])>1):
                if found_type[k][1] == "I":
                    #colorCode = 'chartreuse'
                    colorCode = 'black'
                    mkr = '*'
                elif found_type[k][1] == "b" or found_type[k][1] == "c":
                    #colorCode = 'salmon'
                    colorCode = 'darkblue'
                    mkr = '^'
                elif found_type[k][1] == "a":
                    #colorCode = 'deeppink'
                    colorCode = 'indigo'
                    mkr = 'o'
                else:
                    colorCode = 'magenta'
                    mkr = '>'
            else:
                #colorCode = 'cyan'
                colorCode = 'maroon'
                mkr='s'

            plt.plot(x_coord[k], y_coord[k], marker=mkr, ms=10, color=colorCode, zorder=10)
            plt.text(x_coord[k]-20, y_coord[k]+8, string, fontsize=12, color=colorCode)

        plt.axis('off')
        plt.title(data["Galaxy"][i], fontsize=24, fontweight='bold')
        plt.tight_layout()
        plt.savefig('../Figures/2.MUSE_Galaxy_Images/' + data["Galaxy"][i] + '.png')
        if (len(found_name) > 0):
            plt.savefig('../Figures/2.MUSE_SNe_Galaxy_Images/' + data["Galaxy"][i] + '.png')
        plt.close()

<ipython-input-12-386ab1ee2047>:11: RuntimeWarning: divide by zero encountered in log10
  logmap = np.log10(map)


In [13]:
#Organize the MUSE sample (remove repeat entries and remove those outside of maps)
muselist = list( dict.fromkeys(inMuse) )

inMuseSample = []

for i in range(len(database["Supernova"])):
    #remove by hand the SNe that are just outside of MUSE maps
    if database["Supernova"][i] == "SN2013ej" or database["Supernova"][i] == "SN1979C":
        inMuseSample.append("False")
    elif database["Supernova"][i] in muselist:
        inMuseSample.append("True")
    else: 
        inMuseSample.append("False")
        
# for i in range(len(database)):
#     print(database["Supernova"][i], inMuseSample[i])
    

In [14]:
# Make a table of organized MUSE sample
museSample = Table({"Supernova": database["Supernova"],
                    "InMuseSample" : inMuseSample})

museSample.write("../Data/2.museSample.csv", format = "csv")

In [ ]:
# populate plots and legend labels, gals, sigma, decs, xarr, yarr 

In [ ]:
from matplotlib.patches import Circle
# need to use WCS to find what size the image is and then do a fraction of it 
# convert pixels to 

markerShapes = ["o", "s", "*", "D", "X", "H", "v"]
markerSizes  = [120,120,200,120,120,120,120]

current_cmap = plt.get_cmap("magma")
current_cmap.set_bad("black")

import math
i,j,m=0,0,0
PLOTS_PER_ROW = 3
fig, axs = plt.subplots(math.ceil(len(plots)/PLOTS_PER_ROW),PLOTS_PER_ROW, figsize=(20, 60))
for plot in plots:
    draw_circle = plt.Circle((corner[m], corner[m]),ppb[m], fill = True, color = 'white')

    xArr, yArr = wcss[m].wcs_world2pix(ras[m],decs[m],0)
    logplot = np.log10(plot)
    axs[i][j].imshow(logplot, cmap=current_cmap, aspect="equal", origin = "lower", interpolation = "nearest", vmin = -0.5, vmax = 2.0)
    axs[i][j].set_title(gals[m].upper(), fontsize=20)
    axs[i][j].contour(sigma[m], [3], colors = ["white"], linestyles = ("-"))
    axs[i][j].set_xticks([])
    axs[i][j].set_yticks([])
    axs[i][j].add_patch(draw_circle)
    for n in range(len(decs[m])):
        axs[i][j].scatter(xArr[n], yArr[n], color = "black", marker = markerShapes[n], s = markerSizes[n]+30)
        axs[i][j].scatter(xArr[n], yArr[n], color = "lime", marker = markerShapes[n], s = markerSizes[n], edgecolor="black", label = legendLabels[m][n])
    axs[i][j].legend(loc="best", fontsize=14, handletextpad=0.007)

    j+=1
    if j%PLOTS_PER_ROW==0:
        i+=1
        j=0
    m +=1
plt.tight_layout(pad=0.1, w_pad=0.1, h_pad=0.1)
plt.show()
fig.savefig("../Figures/AllSNeInGalaxies.png", dpi=300)
   

print()